# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [1]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

KeyboardInterrupt: 

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the necessary CSVs to Pandas DataFrames
# YOUR CODE HERE!
avg_housing_units_path = Path("Data/avg_housing_units.csv")
avg_housing_units = pd.read_csv(avg_housing_units_path, index_col="year")

neighborhoods_coordinates_path = Path("Data/neighborhoods_coordinates.csv")
neighborhoods_coordinates = pd.read_csv(neighborhoods_coordinates_path, index_col="Neighborhood")

sfo_neighborhoods_data_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(sfo_neighborhoods_data_path, index_col="year")



- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    # YOUR CODE HERE!
    
    housing_units_plot = sfo_data.hvplot(kind="bar")
    plt.title("SF Housing Units 2010 to 2016")
    plt.xlabel("Year")
    plt.ylabel("Units")
    plt.ylim(350000,390000)
    return housing_units_plot

In [ ]:
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    # YOUR CODE HERE!
    avg_rent = avg_housing_units.groupby('year')['gross_rent'].mean()
    avg_rent_plot = avg_rent.hvplot(y='gross_rent')
    plt.title("Avg Rent by Year")
    plt.xlabel("Year")
    plt.ylabel("Gross Rent")
    return avg_rent_plot

In [ ]:
def average_sales_price():
    """Average Sales Price Per Year."""
    # YOUR CODE HERE!
    avg_sale_price_rent = sfo_data.groupby('year')['sale_price_sqr_foot','gross_rent'].mean()
    avg_sale_price_rent_plot = avg_sale_price_rent.plot(y='sale_price_sqr_foot')
    plt.title("Average Price per SqFt by Year")
    plt.xlabel("Year")
    plt.ylabel("Price per SqFoot")
    return avg_sale_price_rent_plot

In [ ]:
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    # YOUR CODE HERE!
    avg_neighborhood_price = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
    avg_neighborhood_price_plot = avg_neighborhood_price.hvplot(
        x = 'year', 
        y = 'sale_price_sqr_foot_avg', 
        groupby = 'neighborhood', 
        kind = 'line'
    )
    return avg_neighborhood_year_price_plot

In [ ]:
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    # YOUR CODE HERE!
    top10_neighborhoods = avg_neighborhood_price.sort_values('sale_price_sqr_foot_avg', ascending=False,).nlargest(10,'sale_price_sqr_foot_avg').reset_index()
    top10_neighborhoods_plot = top10_neighborhoods.hvplot(
        x='neighborhood',
        y='sale_price_sqr_foot_avg', 
        kind='line'
    )
    return top10_neighborhoods_plot

In [ ]:
def most_expensive_neighborhood_top_ten_year():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""
    avg_neighborhood_year = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
    top10_year = avg_neighborhood_year.sort_values('sale_price_sqr_foot_avg', ascending=False,).nlargest(10,'sale_price_sqr_foot_avg').reset_index()
    top10_year_plot = top10_year.hvplot(
        x='year',
        y=('gross_rent_avg','sale_price_sqr_foot_avg'), 
        groupby = 'neighborhood', 
        kind = 'bar')
    return top10_year_plot

In [ ]:
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    # YOUR CODE HERE!
    parallel_coordinates_plot = px.parallel_coordinates(top10_year, color='sale_price_sqr_foot_avg')
    return parallel_coordinates_plot

In [ ]:
def parallel_categories():
    """Parallel Categories Plot."""
    # YOUR CODE HERE!
    parallel_categories_plot = px.parallel_categories(top10_year, color='sale_price_sqr_foot_avg')
    return parallel_categories_plot

In [ ]:
def neighborhood_map():
    """Neighborhood Map."""
    # YOUR CODE HERE!
    neighborhood_cost = sfo_data.groupby(['neighborhood']).mean().reset_index()
    #neighborhood_cost.rename(columns={"neighborhood":"Neighborhood"}, inplace=True)
    sf_neighborhoods = pd.concat([neighborhood_cost, neighborhoods_coordinates], axis='columns', join='inner')
    sf_map= px.scatter_mapbox(
    sf_neighborhoods,
    lat="Lat",
    lon="Lon",
    color="gross_rent",
    size='sale_price_sqr_foot',
    title="Avg Price Per Sq Ft")
    return sf_map

In [ ]:
def sunburst():
    """Sunburst Plot."""
    # YOUR CODE HERE!
    sf_sunburst=px.sunburst(top10_year, path=['year', 'neighborhood'],values='sale_price_sqr_foot_avg',color='gross_rent_avg')
    return sf_sunburst

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
dashboard_title='# SF Real Estate Analysis'

# Create a tab layout for the dashboard
# YOUR CODE HERE!

tabs = pn.Tabs(
            ("Neighborhood Map", sf_neighborhoods),
            ("Units",avg_housing_units()),
            ("Avg Rent",average_rent()),
            ("AVG Price by Neighborhood", avg_neighborhood_year()),
            ("Top Neighborhoods",pn.Column(top10_year(),avg_neighborhood_year()),
            ("Parrallel Plots",pn.Column(parallel_coordinates_plot, parallel_categories_plot))),
            ("Sunburst Plot", sf_sunburst))
             

In [ ]:
sf_rental_dashboard = pn.Column(dashboard_title,tabs)
sf_rental_dashboard

In [ ]:
## Serve the Panel Dashboard
sf_rental_dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()